In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

In [3]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini")

If we want to create tool manually from a function

In [4]:
from llama_index.core.tools import FunctionTool

multiplytool = FunctionTool.from_defaults(fn=multiply)

FunctionAgent will automatically creates these tools internally

In [5]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    name="calculatorAgent",
    description="Useful for performing basic mathematical operations",
    tools=[multiply, add],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools."
)

In [6]:
result = await agent.run(user_msg="What is 20+(2*4)")
result

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The result of \\( 20 + (2 \\times 4) \\) is 28.')]), structured_response=None, current_agent_name='calculatorAgent', raw={'id': 'chatcmpl-Ctvtojc5YI3LYVHc5Whm350fSjFMY', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1767446640, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_29330a9688', 'usage': None, 'obfuscation': 'dK'}, tool_calls=[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 2, 'b': 4}, tool_id='call_b4BhEiS0PD6497KwGMVwJnM3', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='8')], tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 4}}, raw_output=8, is_error=False), return_direct=False), To

In [9]:
result.response.blocks[0].text

'The result of \\( 20 + (2 \\times 4) \\) is 28.'

Using Existing Tools from llamahub.ai

In [ ]:
pip install llama-index-tools-yahoo-finance

In [11]:
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

finance_tools = YahooFinanceToolSpec().to_tool_list()
finance_tools.extend([multiply, add])

In [12]:
agent = FunctionAgent(
    name="Stock Agent",
    description="Useful for performing financial operations",
    llm=llm,
    tools=finance_tools,
    system_prompt="You are a helpful assistant"
)

response = await agent.run(
    user_msg="what is current stock price of NVIDIA"
)

response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The current stock price of NVIDIA (NVDA) is $188.85.')]), structured_response=None, current_agent_name='Stock Agent', raw={'id': 'chatcmpl-CtvythzLpTSiy3P2gZWox25s2f6KP', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1767446955, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_8bbc38b4db', 'usage': None, 'obfuscation': 'Cq'}, tool_calls=[ToolCallResult(tool_name='stock_basic_info', tool_kwargs={'ticker': 'NVDA'}, tool_id='call_YzR6YfK93vcs8Z5FMRjdrYfi', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='Info: \n{\'address1\': \'2788 San Tomas Expressway\', \'city\': \'Santa Clara\', \'state\': \'CA\', \'zip\': \'95051\', \'co

In [13]:
response.response.blocks[0].text

'The current stock price of NVIDIA (NVDA) is $188.85.'

In [14]:
import sqlite3
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.agent.workflow import ReActAgent

from llama_index.core import SQLDatabase
from IPython.display import Markdown

db_path = "financial_advisor.db"
sqlite_uri = f"sqlite:///{db_path}"

sql_database = SQLDatabase.from_uri(sqlite_uri)

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables = ["customers", "financial_details"]
)

db_tool = QueryEngineTool(
    query_engine=sql_query_engine,
    metadata=ToolMetadata(
        name="financial_database",
        description="""SQL database contains customer financial information.
        Use this to query customer data from customers and financial_details tables."""
    )
)

system_prompt = """You are an experienced Financial Advisor.
Your task is to analyze customer financial data and provide a comprehensive assessment of their financial health.
Make sure to include: current financial health, monthly savings potential, debt-to-income ratio, risk assessment
and key areas for improvement.
Format your response as a structured markdown report with clear sections.
"""

tools = [db_tool]

# We can use ReActAgent if the LLM doesn't support functions/tools else we can use FunctionAgent
# Both are same
agent = ReActAgent(
    name="financial_advisor_agent",
    description="Financial Advisor Agent",
    tools=tools,
    llm=llm,
    system_prompt=system_prompt
)

response = await agent.run(
    """Analyze the financial situation for customer with ID: 2.
    Query the financial _database to get the customer's complete information and provide a 
    comprehensive financial health assessment"""
)

response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Priya Patel, a 42-year-old business owner, has a robust financial profile. Here are the key points of her financial health assessment:\n\n1. **Net Worth**: Priya has a net worth of $2,400,000, indicating a strong financial foundation.\n\n2. **Income and Expenses**: With a monthly salary of $200,000 and monthly expenses of $100,000, she has a positive cash flow of $100,000 each month. This surplus allows her to invest further or save more.\n\n3. **Savings**: Priya currently has savings of $800,000, which provides her with a safety net and the ability to take advantage of investment opportunities.\n\n4. **Investment Style**: Her aggressive investment style suggests she is willing to take risks for potentially higher returns, which can be beneficial given her substantial net worth and cash flow.\n\n5. **Future Planning**: With her next financial revi

In [16]:
Markdown(str(response))

Priya Patel, a 42-year-old business owner, has a robust financial profile. Here are the key points of her financial health assessment:

1. **Net Worth**: Priya has a net worth of $2,400,000, indicating a strong financial foundation.

2. **Income and Expenses**: With a monthly salary of $200,000 and monthly expenses of $100,000, she has a positive cash flow of $100,000 each month. This surplus allows her to invest further or save more.

3. **Savings**: Priya currently has savings of $800,000, which provides her with a safety net and the ability to take advantage of investment opportunities.

4. **Investment Style**: Her aggressive investment style suggests she is willing to take risks for potentially higher returns, which can be beneficial given her substantial net worth and cash flow.

5. **Future Planning**: With her next financial review scheduled for December 26, 2024, it would be prudent for her to assess her investment portfolio and ensure it aligns with her long-term financial goals.

Overall, Priya Patel appears to be in a strong financial position, with good income, manageable expenses, and significant savings. Regular reviews and adjustments to her investment strategy will help maintain and potentially grow her wealth.

In [21]:
agent = FunctionAgent(
    name="Stock Agent",
    description="Useful for performing financial operations",
    llm=llm,
    tools=finance_tools,
    system_prompt="You are a helpful assistant"
)

handler = agent.run(
    user_msg="what is current stock price of Apple"
)

In [19]:
async for event in handler.stream_events():
    print(type(event))

<class 'llama_index.core.agent.workflow.workflow_events.AgentInput'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentOutput'>
<class 'llama_index.core.agent.workflow.workflow_events.ToolCall'>
<class 'llama_index.core.agent.workflow.workflow_events.ToolCallResult'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentInput'>
<class 'llama_index.core.agent.workflow.workflow_events.AgentStream'>
<class 'llama_index.co

In [22]:
from llama_index.core.agent.workflow import AgentStream

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The current stock price of Apple Inc. (AAPL) is $271.01.

AgentWorkflow will use either FunctionAgent or ReactAgent Internally

In [1]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.workflow import Context

# Custom function to set name
async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.store.get("state")
    state["name"] = name
    await ctx.store.set("state", state)
    return f"Name set to {name}"

In [3]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini")

workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="you are a helpful assistant that can set a name",
    #initial_state will be set as state attribute in context
    initial_state={"name": "Sachin"}
)

ctx = Context(workflow)

In [ ]:
resposne = await workflow.run(user_msg="Hi my name is Leela", ctx=ctx)

In [5]:
str(resposne)

'Your name has been updated to Leela.'

In [6]:
ctx.to_dict()

{'version': 1,
 'state': {'state_data': {'_data': {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Current state:\\n{\'name\': \'Sachin\'}\\n\\nCurrent message:\\nHi my name is Leela\\n"}]}, {"role": "assistant", "additional_kwargs": {"tool_calls": [{"index": 0, "id": "call_z6NsDUqCyXdla5JoMDLLTvDg", "function": {"arguments": "{\\"name\\":\\"Leela\\"}", "name": "set_name"}, "type": "function"}]}, "blocks": [{"block_type": "text", "text": ""}, {"block_type": "tool_call", "tool_call_id": "call_z6NsDUqCyXdla5JoMDLLTvDg", "tool_name": "set_name", "tool_kwargs": "{\\"name\\":\\"Leela\\"}"}]}, {"role": "tool", "additional_kwargs": {"tool_call_id": "call_z6NsDUqCyXdla5JoMDLLTvDg"}, "blocks": [{"block_type": "text", "text": "Name set to Leela"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Your name has been updated to Leela